### Install libraries from terminal:

Open your terminal in the project's root directory and run:

`pip install -r requirements.txt` 

This will install all the libraries listed in requirements.txt in your project's environment.

### Install libraries from here

In [48]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Add environment variables.

#### Set up observability with LangSmith, the openai_api_key, and add the user_agent.
In our configuration (.zshrc) file for Zsh (Z Shell), we add:
```bash
export LANGSMITH_TRACING=true
export LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
export LANGSMITH_API_KEY="your-api-key"
export LANGSMITH_PROJECT="ucl_rag_insights"
export OPENAI_USER_AGENT="your-user-agent"
export OPENAI_API_KEY="your-open-ai-api-key"
```



### Import libraries

In [1]:
# Standard Libraries
import os

# Third Party Libraries
from dotenv import load_dotenv
from IPython.display import display
from langchain import hub
from langchain_chroma import Chroma
#from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory

# My Libraries
from helpers import to_markdown, format_docs, get_session_history, load_wikipedia_with_cache, split_text_documents, display_answer

### Show the USER_AGENT environment variable

In [2]:
# Print the USER_AGENT environment variable
user_agent = os.getenv("OPENAI_USER_AGENT")
if user_agent:
    display(to_markdown(f"### UserAgent:\n{user_agent}"))

> ### UserAgent:
> ucl_rag_insights

### Initialize an OpenAI model

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini")

### Load the Documents

In [4]:
titles = [
    "Liga de Campeones de la UEFA",
    #"2021–22 UEFA Champions League",
    "2022–23 UEFA Champions League",  
    "2023–24 UEFA Champions League",
    #"2024–25 UEFA Champions League",
]

docs = []

for title in titles:
    docs.extend(load_wikipedia_with_cache(title))

### Split documents

In [5]:
splits = split_text_documents(
    docs,
    chunk_size=1000,
    chunk_overlap=200
)

### Store documents in Chroma vector database

In [6]:
database = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

### Retrieve documents.

In [7]:
retriever = database.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

### Create the RAG pipeline

In [8]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Use the RAG

In [9]:
question_1 = "Who have won the champions League?"
answer_1 = rag_chain.invoke(question_1)
display_answer(question_1, answer_1)

> #### Who have won the champions League?

> A total of 23 clubs have won the UEFA Champions League/European Cup, with Real Madrid holding the record for the most victories at 15 times. Other notable winners include Ajax, Bayern Munich, AC Milan, Liverpool, and Barcelona. The current champions are Real Madrid, who won the 2024 final against Borussia Dortmund.

### Setup a conversational_rag_chain (Chat History)

In [10]:
CONTEXTUALIZE_Q_SYSTEM_PROMPT = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", CONTEXTUALIZE_Q_SYSTEM_PROMPT),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt
)

SYSTEM_PROMPT = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

display(to_markdown(f"##### SYSTEM_PROMPT: {SYSTEM_PROMPT}"))

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history, # This function updates and retrieves the chat history
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

> ##### SYSTEM_PROMPT: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.
> {context}

### Function uses to answer questions

In [11]:
def answer_this(question, rag_with_memory_chain, key_question="input", key_answer="answer"):
    CONFIG = {
        "configurable": {"session_id": "ucl-rag-insights-session"}
    }
    return rag_with_memory_chain.invoke(
        {key_question: question},
        config=CONFIG,
    )[key_answer]

### Use the conversational RAG

In [12]:
question_2 = "What do we know about Real Madrid?"
answer_2 = answer_this(question_2, conversational_rag_chain)
display_answer(question_2, answer_2)

> #### Store:
> {}

> #### What do we know about Real Madrid?

> Real Madrid are the defending champions of the UEFA Champions League, having won their record-extending 15th title. They also earned a spot to compete in the 2024 UEFA Super Cup and the inaugural edition of the FIFA Intercontinental Cup. They have a strong performance history, winning six titles in eleven seasons.

### Ask another question about the topic

In [33]:
question_3 = "Did Real Madrid win its 15th title?"
answer_3 = answer_this(question_3)
display_answer(question_3, answer_3)

> #### Store:
> {'ucl-rag-insights-session': InMemoryChatMessageHistory(messages=[HumanMessage(content='What do we know about Real Madrid?', additional_kwargs={}, response_metadata={}), AIMessage(content='Real Madrid are the defending champions of the UEFA Champions League, having won their record-extending 15th title in the previous season. They also earned a spot in the 2024 UEFA Super Cup and qualified for the 2025 FIFA Club World Cup. Additionally, they have a successful history in European competitions, winning multiple titles.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Did Real Madrid win its 15th title?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, Real Madrid won their record-extending 15th title in the previous season.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Did Real Madrid earn a spot in the 2024 UEFA Super Cup?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, Real Madrid earned a spot in the 2024 UEFA Super Cup by winning the 2023–24 UEFA Champions League.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What do we know about Real Madrid?', additional_kwargs={}, response_metadata={}), AIMessage(content='Real Madrid is a highly successful football club that recently won its record-extending 15th UEFA Champions League title. They have secured their sixth title in eleven seasons and earned a place in the 2024 UEFA Super Cup, facing Atalanta, as well as a chance to compete in the inaugural FIFA Intercontinental Cup. The club has a rich history in European competitions and has qualified for the 2025 FIFA Club World Cup.', additional_kwargs={}, response_metadata={})])}

> #### Did Real Madrid win its 15th title?

> Yes, Real Madrid won their record-extending 15th title in the previous season.

### Ask another question about the topic

In [34]:
question_4 = "Did Real Madrid earn a spot in the 2024 UEFA Super Cup?"
answer_4 = answer_this(question_4)
display_answer(question_4, answer_4)

> #### Store:
> {'ucl-rag-insights-session': InMemoryChatMessageHistory(messages=[HumanMessage(content='What do we know about Real Madrid?', additional_kwargs={}, response_metadata={}), AIMessage(content='Real Madrid are the defending champions of the UEFA Champions League, having won their record-extending 15th title in the previous season. They also earned a spot in the 2024 UEFA Super Cup and qualified for the 2025 FIFA Club World Cup. Additionally, they have a successful history in European competitions, winning multiple titles.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Did Real Madrid win its 15th title?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, Real Madrid won their record-extending 15th title in the previous season.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Did Real Madrid earn a spot in the 2024 UEFA Super Cup?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, Real Madrid earned a spot in the 2024 UEFA Super Cup by winning the 2023–24 UEFA Champions League.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What do we know about Real Madrid?', additional_kwargs={}, response_metadata={}), AIMessage(content='Real Madrid is a highly successful football club that recently won its record-extending 15th UEFA Champions League title. They have secured their sixth title in eleven seasons and earned a place in the 2024 UEFA Super Cup, facing Atalanta, as well as a chance to compete in the inaugural FIFA Intercontinental Cup. The club has a rich history in European competitions and has qualified for the 2025 FIFA Club World Cup.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Did Real Madrid win its 15th title?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, Real Madrid won their record-extending 15th title in the previous season.', additional_kwargs={}, response_metadata={})])}

> #### Did Real Madrid earn a spot in the 2024 UEFA Super Cup?

> Yes, Real Madrid earned a spot in the 2024 UEFA Super Cup by winning their recent UEFA Champions League title.

### Enhancements Techniques

#### 🚀 Goal:
Improve semantic retrieval by combining BM25 + Embeddings and applying Contextual Compression before passing docs to the LLM.

#### 🛠 Steps We’ll Take:
1️⃣ Hybrid Search → Combine BM25 (keyword-based) + Vector Search (semantic-based. OpenAIEmbeddings + Chroma)<br>
2️⃣ Contextual Compression → Use an LLM to filter irrelevant content<br>
3️⃣ LLMChainExtractor → Extract only key facts from retrieved docs


#### 📝 What This Code Does:
✔ Hybrid Search (BM25 + Vector) → Retrieves keyword + semantic matches<br>
✔ Contextual Compression (LLMChainExtractor) → Filters out irrelevant content<br>
✔ RetrievalQA (GPT-4) → Uses only meaningful retrieved text<br>

🚀 Result? More accurate & cost-efficient RAG system! 🔥

### Import Libraries

In [13]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import RetrievalQA

### Split documents

In [14]:
enhanced_splits = split_text_documents(
    docs,
    chunk_size=400, # Balanced size for all three techniques
    chunk_overlap=50 # Enough to maintain context without redundancy
)

### Store documents in Chroma vector database

In [15]:
enhanced_database = Chroma.from_documents(documents=enhanced_splits, embedding=OpenAIEmbeddings())

### Retrieve documents.

In [16]:
enhanced_retriever = enhanced_database.as_retriever()

### Create BM25 Retriever (Keyword Matching)

In [17]:
bm25_retriever = BM25Retriever.from_documents(enhanced_splits)
bm25_retriever.k = 3  # Limit top 3 keyword-based results

### Combine BM25 + Embeddings using an Ensemble Retriever

In [18]:
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, enhanced_retriever],
    weights=[0.3, 0.7] # The second component has more influence (70%) than the first (30%). 
    # Documents are heavily unstructured and we want to prioritize semantic meaning.
)

### Apply Contextual Compression with LLMChainExtractor

In [19]:
compressor = LLMChainExtractor.from_llm(llm)
compressed_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=hybrid_retriever
)

In [20]:
enhanced_rag_chain = (
    {"context": compressed_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Build the RetrievalQA Chain

In [21]:
enhanced_history_aware_retriever = create_history_aware_retriever(
    llm,
    compressed_retriever,
    contextualize_q_prompt
)

enhanced_rag_chain = create_retrieval_chain(enhanced_history_aware_retriever, question_answer_chain)

# chain_type="map_reduce". Process documents individually, then combine results. Avoid the issue of exceeding token limits for large documents.
# Default value is: chain_type="stuff",  # Stuffing all retrieved docs into the LLM
# Other options: "refine", "map_rerank"
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",  
    retriever=compressed_retriever
)

### Before enhancements

In [22]:
# question_2 = "What do we know about Real Madrid?"

# Before enhancement
display(to_markdown(f"#### Before enhancement"))
display_answer(question_2, answer_2)

> #### Before enhancement

> #### What do we know about Real Madrid?

> Real Madrid are the defending champions of the UEFA Champions League, having won their record-extending 15th title. They also earned a spot to compete in the 2024 UEFA Super Cup and the inaugural edition of the FIFA Intercontinental Cup. They have a strong performance history, winning six titles in eleven seasons.

### After enhancement using enhanced_rag_chain

In [23]:
# Define the conversational RAG chain with history
enhanced_1_conversational_rag_chain = RunnableWithMessageHistory(
    enhanced_rag_chain,  
    get_session_history,  # Updates and retrieves chat history
    input_messages_key="input",  
    history_messages_key="chat_history", 
    output_messages_key="answer" 
)

answer_after_enhancement_1 = answer_this(question_2, enhanced_1_conversational_rag_chain)
display(to_markdown(f"#### After enhancement using enhanced_rag_chain"))
display_answer(question_2, answer_after_enhancement_1)

> #### Store:
> {'ucl-rag-insights-session': InMemoryChatMessageHistory(messages=[HumanMessage(content='What do we know about Real Madrid?', additional_kwargs={}, response_metadata={}), AIMessage(content='Real Madrid are the defending champions of the UEFA Champions League, having won their record-extending 15th title. They also earned a spot to compete in the 2024 UEFA Super Cup and the inaugural edition of the FIFA Intercontinental Cup. They have a strong performance history, winning six titles in eleven seasons.', additional_kwargs={}, response_metadata={})])}

> #### After enhancement using enhanced_rag_chain

> #### What do we know about Real Madrid?

> Real Madrid are a highly successful football club, known for winning a record-extending 15 UEFA Champions League titles. They are the only team whose players have won all available awards in the same season, achieving this in the 2009–10 and 2017–18 seasons. The club is recognized for its competitive history and significant achievements in European football.

# After enhancement using qa_chain

In [126]:
# Define the conversational RAG chain with history
enhanced_2_conversational_rag_chain = RunnableWithMessageHistory(
    qa_chain,  # Function that handles retrieval and QA
    get_session_history,  # Updates and retrieves chat history
    input_messages_key="query",  # Key for input (query)
    history_messages_key="chat_history",  # Key for chat history
    output_messages_key="result"  # Key for the final answer
)

answer_after_enhancement_2 = answer_this(question_2, enhanced_conversational_rag_chain, key_question="query", key_answer="result")
display(to_markdown(f"#### After enhancement using qa_chain"))
display_answer(question_2, answer_after_enhancement_2)

> #### After enhancement using qa_chain

> #### What do we know about Real Madrid?

> Real Madrid are the defending champions, having won their record-extending 15th title the previous season.

### Stop jupyter if needed

In [20]:
# Stop jupyter lab in a specific port
#!jupyter lab stop 8891
